In [1]:
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Lambda, Input, Dense
from tensorflow.keras.losses import mse, binary_crossentropy, kl_divergence
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# UC Irvine 1999 KDD intrusion detection contest
-  [UCI 1999 KDD Intrusion Detection Contest](http://kdd.ics.uci.edu/databases/kddcup99/task.html)
-  Download 10% od the data from the uci.edu archive


In [2]:
urls = [
        "http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz",
        "http://kdd.ics.uci.edu/databases/kddcup99/kddcup.names"
        ]

In [3]:
# this pre-processing code of the KDD dataset is adapter from https://github.com/lironber/GOAD/blob/master/data_loader.py

df_colnames = pd.read_csv(urls[1], skiprows=1, sep=':', names=['f_names', 'f_types'])
df_colnames.loc[df_colnames.shape[0]] = ['status', ' symbolic.']

df = pd.read_csv(urls[0], header=None, names=df_colnames['f_names'].values)
df_symbolic = df_colnames[df_colnames['f_types'].str.contains('symbolic.')]
df_continuous = df_colnames[df_colnames['f_types'].str.contains('continuous.')]
samples = pd.get_dummies(df.iloc[:, :-1], columns=df_symbolic['f_names'][:-1])

labels = np.where(df['status'] == 'normal.', 1, 0)

HTTPError: HTTP Error 403: Forbidden

# Training samples

The **samples** dataframe contains the training samples from the KDD dataset

In [4]:
samples.head()

NameError: name 'samples' is not defined

# The symbolic dataframe

The symbolic dataframe lists the symbolic feature names

In [ ]:
df_symbolic.head()

# The continuous dataframe

The continuous dataframe lists the continous feature names

In [ ]:
df_continuous.head()

# Normalize features using MinMaxScaler

Use [sklearn's MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html) to linearly transform features to range 0:1

In [ ]:
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(samples)

# Ground truth labels
The **labels** indices indicate which samples are __normal__ or __attack__. This would be the ground truth label for supervised learning but here we are using unsupervised learning. We will use the __normal__ examples as our training set because we want a model that is very good at identifying normal network traffic!


In [ ]:
norm_samples = df_scaled[labels == 1]  # normal data
attack_samples = df_scaled[labels == 0]  # attack data

norm_labels = labels[labels == 1]
attack_labels = labels[labels == 0]

In [ ]:
print("There are %6d normal examples (%2.2f%%)." % (norm_samples.shape[0], norm_samples.shape[0]/df_scaled.shape[0]*100.0))
print("There are %6d attack examples (%2.2f%%)." % (attack_samples.shape[0], attack_samples.shape[0]/df_scaled.shape[0]*100.0))

# Training set
The training set will consist of 80% of the normal examples. The rest are used as the validation set.

In [ ]:
len_norm = len(norm_samples)
len_norm_train = int(0.8 * len_norm)
X_train = norm_samples[:len_norm_train]

# Test set 
The test set consists of 50% attack and 50% normal examples. These examples have never been seen by the model. This is very important for integrity of evaluating our model. The test set is used to evaluate the performance of our anomaly detector. We build it by taking the unused 20% of our normal examples plus an equal number of attack examples (the last N attack examples). The test set has labels, 0 for _attack_ and 1 for _normal_.


In [ ]:
X_test_norm = norm_samples[len_norm_train:]
len_attack_test = len(X_test_norm) # we will use the same number
X_test_attack = attack_samples[:len_attack_test]

X_test = np.concatenate([X_test_norm, X_test_attack])
y_test = np.ones(len(X_test))
y_test[:len(X_test_norm)] = 0

In [ ]:
print("There are %d training examples and %d test examples." % (X_train.shape[0], X_test.shape[0]))

# Reconstruction Error

Compute the Mean Absolute Error between two values. 
Typically, one input is predicted (output from the decoder) and the other is a known value.
The MAE is the Cartesian distance between the two values.

In [ ]:
def get_error_term(v1, v2, _rmse=True):
    if _rmse:
        return np.sqrt(np.mean((v1 - v2) ** 2, axis=1))
    #return MAE
    return np.mean(abs(v1 - v2), axis=1)

# The reparameterization trick

The **sample** function computes a parameter value based on a normal distribution about a mean. In this case the mean and the variance of the distribution are learned by the model. This function outputs a continuous hidden variable (latent) value that is the output of the encoder and the input to the decoder.

In [ ]:
def sample(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

# Encoder/Decoder information bottleneck

An Encoder is formed by successive layers with fewer and fewer units. This is called an information bottleneck. 
The reduction in information dimensionality forces the network to learn (only) the most important features of the input.

The reverse is done for the decoder. Layers with increasing numbers of units are added in succession.
The final layer output is the same dimension as the input, enforcing a decoder.

To summarize, the **encoder** learns to represent the input as a set of latent variables that are each the output of a normal probability distribution. The **decoder** learns to produce a valid input from a set of latent variables.

# Autoencoder dimensionality

We guess at a dimensions by adding an intermediate layer with half the size as the input.
The latent space is a third of the input size.

In [ ]:
original_dim = X_train.shape[1]
input_shape = (original_dim,)
intermediate_dim = int(original_dim / 2)
latent_dim = int(original_dim / 3)

# Encoder

We use Keras to build the Encoder.

In [ ]:
# encoder model
inputs = Input(shape=input_shape, name='encoder_input')
x = Dense(intermediate_dim, activation='relu')(inputs)
z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)
# use the reparameterization trick and get the output from the sample() function
z = Lambda(sample, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])
encoder = Model(inputs, z, name='encoder')
encoder.summary()

# Decoder

We use Keras to build the Decoder.

In [ ]:
# decoder model
latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
x = Dense(intermediate_dim, activation='relu')(latent_inputs)
outputs = Dense(original_dim, activation='sigmoid')(x)
# Instantiate the decoder model:
decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()

# Autoencoder

We assemble the full network by combining the Encoder and Decoder into an Autoencoder.

In [ ]:
# full VAE model
outputs = decoder(encoder(inputs))
vae_model = Model(inputs, outputs, name='vae_mlp')

# Objective Function

To train the model we need a training objective. This is a differentiable function that is called the __loss function__.
In the case of a VAE we combine the reconstruction loss with the KL divergence as our loss function.
The reconstruction loss is simply the mean squared error of the input to the encoder compared to the output of the decoder. This measures the quality of the output, how well does it match the input. 

The KL divergence measures the distance between the assumed normal distribution (assumed ground truth) and the distribution learned by the network in the variable z_log_var.

In [ ]:
# the KL loss function:
def vae_loss(x, x_decoded_mean):
    # compute the average MSE error, then scale it up, ie. simply sum on all axes
    reconstruction_loss = K.sum(K.square(x - x_decoded_mean))
    # compute the KL loss
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.square(K.exp(z_log_var)), axis=-1)
    # return the average loss over all 
    total_loss = K.mean(reconstruction_loss + kl_loss)    
    #total_loss = reconstruction_loss + kl_loss
    return total_loss

# Training Loop

We use the Adam optimizer to perform gradient descent optimization over the loss function to train the model.
We use 32 epochs of trainging and a batch size of 256. These values are hyperparameters of the model. 
They can be adjusted (tuned) to improve the model, but only so far before it gets worse.

In [ ]:
opt = optimizers.Adam(learning_rate=3e-5, clipvalue=0.5)
#opt = optimizers.RMSprop(learning_rate=0.0001)

vae_model.compile(optimizer=opt, loss=vae_loss)
vae_model.summary()
# Finally, we train the model:
results = vae_model.fit(X_train, X_train,
                        shuffle=True,
                        epochs=40,
                        batch_size=64)

We can visualize gradient descent by plotting each epoch's loss.

In [ ]:
plt.plot(results.history['loss'])
#plt.plot(results.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right');
plt.show()

In [ ]:
X_train_pred = vae_model.predict(X_train)

# Reconstruction error

Now that we trained our model we want to see what the typical error is when we used the autoencoder on the training examples it has seen.
We call the __get_error_term__ function to get the MAE for each training example and our model's corresponding prediction.

We use the top percentile reconstruction error value as the normal/attack boundary threshold. 
We will use this error threshold value to make predictions on new samples as to whether they are normal (reconstruction error is less than or equal to the threshold) or attack (greater than the threshold).

In [ ]:
mae_vector = get_error_term(X_train_pred, X_train, _rmse=False)
print(f'Avg error {np.mean(mae_vector)}\nmedian error {np.median(mae_vector)}\n99Q: {np.quantile(mae_vector, 0.99)}')
max_error = np.max(mae_vector)
print(f'Max error {max_error}')
q99_error = np.quantile(mae_vector, 0.99)
error_thresh = q99_error * 1.5
print(f'setting threshold at { error_thresh } ')


# Testing the anomaly detector

Now it's time to use the autoencoder and the reconstruction error threshold to make predictions on samples we've never observed.
We use the test set that we prepared earlier. It is composed from normal examples that were not used for training and an equal number of attack samples.

As before, we make feed the examples to the autoencoder and get the reconstruction error for each examples. We declare as **anomalies** all the examples where the reconstruction error is greater than the threshold.

In [ ]:
X_pred = vae_model.predict(X_test)
mae_vector = get_error_term(X_pred, X_test, _rmse=False)
anomalies = (mae_vector > error_thresh)

np.count_nonzero(anomalies) / len(anomalies)

# It works!
We find roughly 44% of the test examples to be **attack** samples. Ideally this would have been 50% but it's not too bad!

# Classification Report

We can use the [sklearn classification_report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) to get a more completed evaluation.
The classification report can be used for binary classification as is the case here.
**Precision** measures how well we identify each class without making mistakes.
**Recall** measures how well we do at finding all the examples of each class.

You can see that we have 99% confidence that when we predict it's normal, it is. That confidence comes at a cost though.
We only find 85% of the normal samples. 
On the other hand (this is typical) we have 87% confidence that when we say it's an attack it is and we recover 99% of the malicious examples.

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, anomalies))

In [ ]:
X_pred.shape

# Exploring latent space

We can visualize the latent variables that the autoencoder learned and uses to produce the predictions.
We do obtain the latend variables by feeding examples through the encoder without decoding them.

In [ ]:
X_encoded = encoder.predict(X_test)

# Principle Component Analysis - PCA

We use [sklearn's PCA tool](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) to find the principle components of the latent variables.
We choose to use 2 components here but you could choose more.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=10)
X_transform = pca.fit_transform(X_encoded)

In [ ]:
plt.figure(figsize=(12, 10))
sns.scatterplot(x=X_transform[:, 0], y=X_transform[:, 1], s=20, hue=mae_vector)
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(12, 10))
sns.scatterplot(x=X_transform[:, 0], y=X_transform[:, 1], s=20, hue=anomalies)
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(12, 10))
sns.scatterplot(x=X_transform[:, 0], y=X_transform[:, 1], s=10, hue=y_test)
plt.grid()
plt.show()

# Save the model for later use

Using keras we can save the trained model's weights and load it later

In [ ]:
#vae_model.save('vae_mlp.h5')

In [ ]:
vae_mlp = load_model('vae_mlp.h5', custom_objects={'vae_loss': vae_loss})


In [ ]:
_X_pred = vae_model.predict(X_test)
_mae_vector = get_error_term(_X_pred, X_test, _rmse=False)
_anomalies = (_mae_vector > error_thresh)

np.count_nonzero(_anomalies) / len(_anomalies)

# TODO
Figure out how to retrieve the encoder from the loaded model.

In [ ]:
layer = vae_mlp.get_layer('encoder')

In [ ]:
_X_encoded = layer.predict(X_test)
pca = PCA(n_components=10)
_X_transform = pca.fit_transform(_X_encoded)

In [ ]:
plt.figure(figsize=(12, 10))
sns.scatterplot(x=_X_transform[:, 0], y=_X_transform[:, 1], s=10, hue=y_test)
plt.grid()
plt.show()